In [ ]:
import numpy as np
import pandas as pd
import nd2reader
import matplotlib.pyplot as plt
import holoviews as hv
import skimage.filters
import skimage.feature
import scipy.ndimage
import peakutils
from tqdm import tnrange, tqdm_notebook
import dask
import dask.array as da
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from cytoolz import partial, compose
from itertools import repeat
from glob import glob
import cachetools
import numpy_indexed
import pickle
import warnings
import os
from numbers import Integral
from dask.delayed import Delayed

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from segmentation import *
# from util import *
# from matriarch_stub import *
import segmentation
import matriarch_stub

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)
hv.extension("bokeh")

# Run

In [ ]:
dask.config.config["distributed"]["scheduler"]["allowed-failures"] = 20
# dask.config.config['distributed']['worker']['memory'] = {'target': 0.4,
#                                                         'spill': 0.5,
#                                                         'pause': 0.9,
#                                                         'terminate': 0.95}

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="03:00:00",
    memory="8GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/projects/molecule-counting/log",
    cores=1,
    processes=1,
)
# diagnostics_port=('127.0.0.1', 8787),
# env_extra=['export PYTHONPATH=\"/home/jqs1/projects/matriarch\"'])
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(0)

## Run

In [ ]:
for filename in fluorescence_filenames[2:4]:
    frame = segmentation.get_nd2_frame(filename, 0, 0, 0)
    plt.figure()
    plt.imshow(frame)
    print("&", frame.max(), frame.min(), frame.dtype)
    # labels = segmentation.segment(frame)
    # frame = frame.astype(float)
    # frame = (frame - dark_frame) / (flat_fields['FITC'] - dark_frame)
    plt.figure()
    plt.imshow(frame)
    print(">>>", frame.max(), frame.min(), frame.dtype)
    labels = segmentation.segment(frame)
    plt.figure()
    plt.imshow(labels)
    print(">", labels.max())

In [ ]:
frame - frame.min()

In [ ]:
frame = segmentation.get_nd2_frame(fluorescence_filenames[2], 0, 0, 0)

In [ ]:
flat = flat_fields["FITC"]

In [ ]:
flat.mean()

In [ ]:
(frame / flat).max()

In [ ]:
plt.hist((frame / flat.mean()).flat, bins=100)

In [ ]:
plt.hist(frame.flat, bins=100)

In [ ]:
segmentation.segment((frame / flat.mean())[:300, :300], diagnostics=diag2).max()

In [ ]:
img_blurred = diag["img_blurred"].data
img2_blurred = diag2["img_blurred"].data

In [ ]:
plt.imshow(
    skimage.filters.ridges.compute_hessian_eigenvalues(img_blurred, 1, sorting="abs")[0]
    != skimage.filters.ridges.compute_hessian_eigenvalues(
        img_blurred, 1, sorting="val"
    )[0]
)

In [ ]:
abs_k1, abs_k2 = skimage.filters.ridges.compute_hessian_eigenvalues(
    img_blurred, 1.5, sorting="abs"
)

In [ ]:
val_k1, val_k2 = skimage.filters.ridges.compute_hessian_eigenvalues(
    img_blurred, 1.5, sorting="val"
)

In [ ]:
none_k1, none_k2 = skimage.filters.ridges.compute_hessian_eigenvalues(
    img_blurred, 1.5, sorting="none"
)

In [ ]:
def hess(img):
    I = skimage.filters.gaussian(img, 1.5)
    I_x = skimage.filters.sobel_h(I)
    I_y = skimage.filters.sobel_v(I)
    I_xx = skimage.filters.sobel_h(I_x)
    I_xy = skimage.filters.sobel_v(I_x)
    I_yx = skimage.filters.sobel_h(I_y)
    I_yy = skimage.filters.sobel_v(I_y)
    kappa_1 = (I_xx + I_yy) / 2
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", RuntimeWarning)
        kappa_2 = (np.sqrt((I_xx + I_yy) ** 2 - 4 * (I_xx * I_yy - I_xy * I_yx))) / 2
    k1 = kappa_1 + kappa_2
    k2 = kappa_1 - kappa_2
    k1[np.isnan(k1)] = 0
    k2[np.isnan(k2)] = 0
    return k1, k2

In [ ]:
my_k1, my_k2 = hess(img_blurred.T)

In [ ]:
plt.imshow(abs_k1[:300, :300])

In [ ]:
plt.imshow(val_k1[:300, :300])

In [ ]:
plt.imshow(none_k2[:300, :300])

In [ ]:
plt.imshow(my_k2[:300, :300])

In [ ]:
(val_k1[:300, :300] - none_k2[:300, :300]).std()

In [ ]:
plt.imshow(k2[:300, :300])

In [ ]:
plt.imshow(k2[:300, :300])

In [ ]:
img_k1 = diag["img_k1"].data
img2_k1 = diag2["img_k1"].data

In [ ]:
plt.hist(img_k1.flat, log=True, bins=100)

In [ ]:
plt.hist(img2_k1.flat, log=True, bins=100)

In [ ]:
diag = {}
labels = segmentation.segment(frame, diagnostics=diag)
diag2 = {}
labels2 = segmentation.segment(frame / flat, diagnostics=diag2)

In [ ]:
l = "img_k1"
diag[l] + diag2[l].redim(z="z2")

In [ ]:
plt.imshow(labels2)

## Old

In [ ]:
funcs = {"mean": np.mean}  # ,
#'median': np.median}
#'p0.05': partial(np.percentile, q=5),
#'p0.20': partial(np.percentile, q=20),
#'p0.70': partial(np.percentile, q=70),
#'p0.95': partial(np.percentile, q=95)}

In [ ]:
base_filename = "/n/scratch2/jqs1/fidelity/190313"
fluorescence_filenames = (
    glob(os.path.join(base_filename, "fluorescence/*.nd2"))
    + glob("/n/scratch2/jqs1/fidelity/190325/fluorescence/*/*.nd2")
    + glob("/n/scratch2/jqs1/fidelity/190326/*.nd2")
)
phase_filenames = glob(os.path.join(base_filename, "phase/*_0001.nd2")) + glob(
    "/n/scratch2/jqs1/fidelity/190325/phase/*/*_0001.nd2"
)
sandwich_filenames = glob(os.path.join(base_filename, "sandwich/*_0001.nd2"))

In [ ]:
dark_frames = segmentation.nd2_to_dask(
    os.path.join(base_filename, "calibration/dark_100ms.nd2"), 0, 0
)
dark_frame = dark_frames.mean(axis=0)
# TODO: hack
# dark_frame = dark_frame.compute()
# dark_frame = client.persist(dark_frame)
# dark_frame = client.scatter(dark_frame, broadcast=True)
dark_frame = dark_frame.to_delayed()[0, 0]

In [ ]:
flat_fields = {}
for filename in glob(os.path.join(base_filename, "calibration/*flat*100ms*.nd2")):
    channel = segmentation.get_nd2_reader(filename).metadata["channels"][0]
    flat_field = segmentation.nd2_to_dask(filename, 0, 0).mean(axis=0)
    # TODO: hack
    # flat_field = flat_field.compute()
    # flat_field = client.scatter(flat_field, broadcast=True)
    # flat_field = client.persist(flat_field)
    flat_field = flat_field.to_delayed()[0, 0]
    flat_fields[channel] = flat_field

In [ ]:
data_graph = {}
for photobleaching_filename in fluorescence_filenames[:0]:
    data_graph[photobleaching_filename] = segmentation.process_file(
        funcs, photobleaching_filename, dark_frame=dark_frame, flat_fields=flat_fields
    )

for photobleaching_filename in phase_filenames[:0]:
    segmentation_filename = photobleaching_filename.replace("_0001.nd2", ".nd2")
    data_graph[segmentation_filename] = segmentation.process_file(
        funcs,
        photobleaching_filename,
        segmentation_filename=segmentation_filename,
        dark_frame=dark_frame,
        flat_fields=flat_fields,
    )

for initial_filename in sandwich_filenames[:0]:
    segmentation_filename = initial_filename.replace("_0001.nd2", ".nd2")
    photobleaching_filename = initial_filename.replace("_0001.nd2", "_0002.nd2")
    final_filename = initial_filename.replace("_0001.nd2", "_0003.nd2")
    data_graph[segmentation_filename] = segmentation.process_file(
        funcs,
        photobleaching_filename,
        segmentation_filename=segmentation_filename,
        initial_filename=initial_filename,
        final_filename=final_filename,
        dark_frame=dark_frame,
        flat_fields=flat_fields,
    )

In [ ]:
# data_futures = matriarch_stub.recursive_map(lambda x: client.compute(x), data_graph, shortcircuit=Delayed)

In [ ]:
# split up computes so we can gather results from multiple workers
# (otherwise the single worker assembling the dict will run out of memory)
data_futures = {k: client.compute(v) for k, v in data_graph.items()}
data_futures["_calibration"] = {"dark_frame": dark_frame, "flat_fields": flat_fields}

In [ ]:
from dask.distributed import Future

In [ ]:
matriarch_stub.recursive_map(lambda x: client, data_futures, shortcircuit=dask.)

In [ ]:
del data_futures, data_graph

In [ ]:
client.restart()

## Save data

In [ ]:
hv.Image(
    data[
        "/n/scratch2/jqs1/fidelity/190313/fluorescence/190313_mkate_100ms_50pct_laser.nd2"
    ][0]["labels"]
)

In [ ]:
data = client.gather(data_futures)

In [ ]:
data.keys()

In [ ]:
with open("190326photobleaching_flatcorr.pickle", "wb") as f:
    pickle.dump(data, f)

In [ ]:
# time: 96593s
# 65% map_over_labels, 30% get_nd2_frame
# time: 109885s (3/22)
# 56% map_over_labels 43% get_nd2_frame
# time: 83334s (3/26, only fluor)
# 88% map_over_labels 9% get_nd2_frame

In [ ]:
%%output size=200
hv.HoloMap(
    {
        filename: RevImage(permute_labels(data[filename][0]["labels"]))
        for filename in data.keys()
    }
)

# Plotting

In [ ]:
with open("190311photobleaching.pickle", "rb") as f:
    data = pickle.load(f)

In [ ]:
list(trace_res.keys())

In [ ]:
b = trace_res["/n/scratch2/jqs1/fidelity/190311/190311_205a_10ms_laser10pct_001.nd2"]

In [ ]:
plt.plot(np.log(b[0]["mean"][10]))

In [ ]:
b[1].hist("area", bins=100)

In [ ]:
traces = z["/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2"][0][
    "traces"
]["mean"]

In [ ]:
traces = b[0]["mean"]

In [ ]:
idxs = np.random.permutation(traces.shape[0])

In [ ]:
%%output size=250
curves = [
    {
        "x": np.arange(traces.shape[1]),
        "y": np.log(traces[i] / traces[i, 0]),
        "i": idxs[i],
    }
    for i in range(traces.shape[0] // 10)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20")

In [ ]:
%%output size=250
hv.Path((np.arange(traces.shape[1]), traces.T))

In [ ]:
%%output size=250
hv.Overlay.from_values([hv.Curve(t) for t in traces])

In [ ]:
%%output size=250
hv.Image(img) + hv.Image(segment(img))